# Worksheet 08

Name:  Deep Patel

UID: U37068394

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [5]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[5.513848719635686, 4.407431770153509, 5.086566060601904, 3.978649451384015, 5.533377655157152, 5.172070402742254, 4.0828846098321705, 5.041368308870558, 4.408307269624098, 5.07946918402809]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [3]:
c2 = np.random.normal(8 , 1 , 10 ).tolist()
print(c2)

[5.504251569512027, 6.751644782086487, 6.92666325954305, 8.036226880191286, 8.559854523419823, 7.542150718566511, 7.95337676892603, 9.422561158970067, 10.306014363403692, 7.333585724815152]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [6]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[5.07946918402809, 4.408307269624098, 9.422561158970067, 5.041368308870558, 4.0828846098321705, 5.172070402742254, 5.533377655157152, 7.95337676892603, 7.542150718566511, 3.978649451384015]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Mixture Model:
* X comes from a mixture model with k mixture components if the prob distribution of X is $P(X = x) = \sum_{j=1}^{k}P(C_j)P(X = x | C_j)$

For each individual probability, mean & variance

And a probaility of x being i-th species 

**Gaussian Mixture Model**: when all $P(X | S_j)$ are normal distribution with parameters: mean & standard deviation

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [8]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] != 0, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2)  ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[0])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.07946918402809, 4.408307269624098, 5.041368308870558, 4.0828846098321705, 5.172070402742254, 5.533377655157152, 3.978649451384015]
[9.422561158970067, 7.95337676892603, 7.542150718566511]
P(C_1) = 0.7,  P(C_2) = 0.3
mean_1 = 4.7565895545197625,  mean_2 = 8.306029548820868
var_1 = 0.30596342702115203,  var_2 = 13.250030168791136


/Users/deeppatel/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


means are close but variance is very different

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [9]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  5.07946918402809
probability of observing that point if it came from cluster 0 =  0.7471677576333902
probability of observing that point if it came from cluster 1 =  0.029229179839297195
point =  4.408307269624098
probability of observing that point if it came from cluster 0 =  0.6821350724125215
probability of observing that point if it came from cluster 1 =  0.028833842670368273
point =  9.422561158970067
probability of observing that point if it came from cluster 0 =  4.11433990529922e-51
probability of observing that point if it came from cluster 1 =  0.03000207386268916
point =  5.041368308870558
probability of observing that point if it came from cluster 0 =  0.8455173138314742
probability of observing that point if it came from cluster 1 =  0.02920859908230604
point =  4.0828846098321705
probability of observing that point if it came from cluster 0 =  0.11545667139618093
probability of observing that point if it came from cluster 1 =  0.02861764267235044
point =  5.1720

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [10]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), 
        sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]
var = [sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_c0_x, data)]) / sum(prob_c0_x),
       sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.6443893784306722,  P(C_2) = 0.3556106215693278
mean_1 = 4.764219046646771,  mean_2 = 7.73714066945386
var_1 = 0.28778162080437436,  var_2 = 2.3749847242114344


different. It's more likely to be classified as C2

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [11]:
prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  5.07946918402809
probability of observing that point if it came from cluster 0 =  0.7607963033692714
probability of observing that point if it came from cluster 1 =  0.0898117024747253
point =  4.408307269624098
probability of observing that point if it came from cluster 0 =  0.6452281786309024
probability of observing that point if it came from cluster 1 =  0.0629003742462504
point =  9.422561158970067
probability of observing that point if it came from cluster 0 =  1.7564566176584518e-57
probability of observing that point if it came from cluster 1 =  0.13058448605866974
point =  5.041368308870558
probability of observing that point if it came from cluster 0 =  0.871864061519051
probability of observing that point if it came from cluster 1 =  0.08820243132854257
point =  4.0828846098321705
probability of observing that point if it came from cluster 0 =  0.08407823842570128
probability of observing that point if it came from cluster 1 =  0.05142441294057215
point =  5.1720704

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [12]:
assignments = [[], []]
for x in zip(data, prob_c0_x):
    p = x[1]
    d = x[0]
    if random.random() <= p:
        assignments[0].append(d)
    else:
        assignments[1].append(d)
print(f"points in cluster 0 : {assignments[0]}\npoints in cluster 1 : {assignments[1]}")

points in cluster 0 : [5.07946918402809, 4.408307269624098, 5.041368308870558, 4.0828846098321705, 5.172070402742254, 3.978649451384015]
points in cluster 1 : [9.422561158970067, 5.533377655157152, 7.95337676892603, 7.542150718566511]
